In [50]:
#@title GET THE CALIBERATION DATA, WE FOCUS ON Q MATRIX
#run
import numpy as np
import cv2

def parse_calibration_and_compute_Q(filepath):
    """Parse KITTI calibration file and compute Q matrix."""
    data = {}
    with open(filepath, 'r') as f:
        for line in f.readlines():
            if ':' in line:
                key, value = line.split(':')
                data[key] = np.array([float(x) for x in value.split()])

    # Extract camera parameters
    P2 = data['P2'].reshape(3, 4)
    P3 = data['P3'].reshape(3, 4)
    f = P2[0, 0]
    cx, cy = P2[0, 2], P2[1, 2]
    Tx = P3[0, 3] / (-f)

    # Compute Q matrix
    Q = np.zeros((4, 4))
    Q[0, 0] = Q[1, 1] = 1
    Q[0, 3] = -cx
    Q[1, 3] = -cy
    Q[2, 3] = f
    Q[3, 2] = -1 / Tx
    Q[3, 3] = (cx - P3[0, 2]) / Tx  # cx' - cx

    return data, Q


calib_data, Q_matrix = parse_calibration_and_compute_Q('DATA/CALIB/000029.txt')
print("Calibration data:")
print(calib_data)
print("\nQ matrix:")
print(Q_matrix)


Calibration data:
{'P0': array([721.5377,   0.    , 609.5593,   0.    ,   0.    , 721.5377,
       172.854 ,   0.    ,   0.    ,   0.    ,   1.    ,   0.    ]), 'P1': array([ 721.5377,    0.    ,  609.5593, -387.5744,    0.    ,  721.5377,
        172.854 ,    0.    ,    0.    ,    0.    ,    1.    ,    0.    ]), 'P2': array([7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01,
       0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01,
       0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]), 'P3': array([ 7.215377e+02,  0.000000e+00,  6.095593e+02, -3.395242e+02,
        0.000000e+00,  7.215377e+02,  1.728540e+02,  2.199936e+00,
        0.000000e+00,  0.000000e+00,  1.000000e+00,  2.729905e-03]), 'R0_rect': array([ 0.9999239 ,  0.00983776, -0.00744505, -0.0098698 ,  0.9999421 ,
       -0.00427846,  0.00740253,  0.00435161,  0.9999631 ]), 'Tr_velo_to_cam': array([ 7.533745e-03, -9.999714e-01, -6.166020e-04, -4.069766e-03,
        1.480249e-02,  7.280733e-04, -9.998902e-01

In [77]:
import os

# Function to process a pair of left and right images and generate point cloud
def process_image_pair(left_img_path, right_img_path, output_folder):
    # Load left and right images
    imgL = cv2.imread(left_img_path)
    imgR = cv2.imread(right_img_path)

    # Downsample images
    imgL = downsample_image(imgL, 3)
    imgR = downsample_image(imgR, 3)

    imgLgray = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgRgray = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    # Compute disparity map
    disparity_map = stereo.compute(imgLgray, imgRgray)

    # Generate point cloud from disparity map
    points_3D = cv2.reprojectImageTo3D(disparity_map, Q_matrix, handleMissingValues=False)
    colors = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
    mask_map = disparity_map > disparity_map.min()
    output_points = points_3D[mask_map]
    output_colors = colors[mask_map]

    # Define output file path
    filename = os.path.splitext(os.path.basename(left_img_path))[0]
    output_file = os.path.join(output_folder, f"pointCloud_{filename}.ply")

    # Generate point cloud file
    create_point_cloud_file(output_points, output_colors, output_file)

# Paths to left and right image folders
left_folder = "DATA/LEFT/"
right_folder = "DATA/RIGHT"

# Output folder for point clouds
output_folder = "point_clouds_output"

# Iterate over images in the folders
left_images = sorted(os.listdir(left_folder))
right_images = sorted(os.listdir(right_folder))
for left_img_name, right_img_name in zip(left_images, right_images):
    left_img_path = os.path.join(left_folder, left_img_name)
    right_img_path = os.path.join(right_folder, right_img_name)
    process_image_pair(left_img_path, right_img_path, output_folder)

print("Point clouds generated successfully.")


Point clouds generated successfully.


In [79]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import cv2

opencv_pcd_path = "point_clouds_output/pointCloud_000029L.ply"
pcd = o3d.io.read_point_cloud(opencv_pcd_path)

# # Flip it, otherwise the pointcloud will be upside down
# pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Flip it horizontally
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])
